## Introduction
In order to do Data Science we have to be able to gather data. Oftentimes we get data from an API, sometimes from a database. This Jupyter notebook we'll guide you through a project that will involve getting data from an API, storing it in a relational database and performing some SQL queries to do some basic data analysis.

Our API of choice for this project will be [GitHub Jobs](https://jobs.github.com). Besides being simple to use, it could be a useful resource to job-hunt. 

## Assingment
- Get acquainted with the GitHub Jobs web interface and read through their API page.
- Import the Requests package and call the API with the parameters of your choice.
- Print the status code to make sure it went well.

In [1]:
import requests

url = "https://jobs.github.com/positions.json"
search_term = 'data_science'

params = {'description': search_term}
response = requests.get(url, params=params)


In [2]:
import json

In [3]:
print(json.dumps(response.json(), sort_keys=True,indent=4))

[
    {
        "company": "Leadpages",
        "company_logo": "http://github-jobs.s3.amazonaws.com/012e2336-d389-11e6-867c-d25a8523294d.png",
        "company_url": "http://www.leadpages.net/",
        "created_at": "Thu Jan 05 21:04:49 UTC 2017",
        "description": "<p>We promise you haven\u2019t worked with anyone <a href=\"http://www.leadpages.net/careers\">like us</a> before.  We\u2019re a Minneapolis / St. Paul, MN based startup that\u2019s changing the way businesses do marketing, lead generation, and conversion rate optimization online.  With a growing team that works closely together, we move fast and enjoy what we do. </p>\n\n<p>As a <strong>Cloud Engineer / Site Reliability Engineer</strong>, you&#39;ll work with a dynamic team to help build out and manage our cloud infrastructure and deployment pipelines. You\u2019ll partner with development teams to build and support tooling - specializing in centralized logging, monitoring and alerting. You will also develop a strong

## Assignment
- Import json and print the whole json response with the json.dumps() function. See what kind of data it contains.
- Get the dictionary of the first job and call the method keys() to have a better idea of the data at hand.
- Print some of the first job's content, including images and HTML. You might need to import the IPython package.

In [4]:
json_response = response.json()
first_job = json_response[0]
first_job.keys()

dict_keys(['id', 'created_at', 'title', 'location', 'type', 'description', 'how_to_apply', 'company', 'company_url', 'company_logo', 'url'])

In [5]:
from IPython.display import Image
image_url = first_job['company_logo']
Image(url=image_url)

In [6]:
from IPython.core.display import display, HTML

display(HTML(first_job['description']))

In [7]:
display(HTML(first_job['how_to_apply']))

## Assignment
- Import SQLAlchemy and configure a SQLite database.
- Create 4 tables: company, job, location, github_metadata. Pay sppecial attention on primary and foreign keys as well as unique and non-nullable columns.
- Make sure you store 'created_at' as a datetime and not a string.

In [8]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey, DateTime

engine = create_engine('sqlite:///:memory:', echo=True)
metadata = MetaData()

company_table = Table('company', metadata,
                      Column('id', Integer, primary_key=True),
                      Column('name', String, nullable=False),
                      Column('url', String, unique=True),
                      Column('logo_url', String))

job_table = Table('job', metadata,
                  Column('id', Integer, primary_key=True),
                  Column('company_id', None, ForeignKey('company.id'), nullable=False),
                  Column('type_', Integer),
                  Column('title', String, nullable=False),
                  Column('description', String),
                  Column('how_to_apply', String))

location_table = Table('location', metadata,
                      Column('job_id', None, ForeignKey('job.id'), primary_key=True),
                      Column('description', String),
                      Column('remote_ok', Integer))

github_metadata_table = Table('github_metadata', metadata,
                              Column('job_id', None, ForeignKey('job.id'), primary_key=True),
                              Column('github_id', String, unique=True),
                              Column('url', String, nullable=False),
                              Column('created_at', DateTime, nullable=False))

metadata.create_all(engine)
conn = engine.connect()


2017-01-06 11:48:36,198 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-01-06 11:48:36,199 INFO sqlalchemy.engine.base.Engine ()
2017-01-06 11:48:36,202 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-01-06 11:48:36,204 INFO sqlalchemy.engine.base.Engine ()
2017-01-06 11:48:36,205 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("company")
2017-01-06 11:48:36,206 INFO sqlalchemy.engine.base.Engine ()
2017-01-06 11:48:36,208 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("job")
2017-01-06 11:48:36,209 INFO sqlalchemy.engine.base.Engine ()
2017-01-06 11:48:36,211 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("location")
2017-01-06 11:48:36,211 INFO sqlalchemy.engine.base.Engine ()
2017-01-06 11:48:36,213 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("github_metadata")
2017-01-06 11:48:36,214 INFO sqlalchemy.engine.base.Engine ()
2017-01-06 11:48:36,216 INFO sqla

In [9]:
from enum import Enum

class JobType(Enum):
    full_time = 1
    part_time = 2

    @staticmethod
    def from_string(string):
        if string.lower() == 'full time' or string.lower() == 'full-time':
            return JobType.full_time
        elif string.lower() == 'part time' or string.lower() == 'part-time':
            return JobType.part_time
        else:
            raise RuntimeError('JobType not supported: {}'.format(string))

In [10]:
def insert_company(name, url, logo_url, connection=conn):
    """Inserts a new entry in company_table and returns its id."""
    insert = company_table.insert().values(name=name, url=url, logo_url=logo_url)
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

In [11]:
def insert_job(company_id, type_str, title, description, how_to_apply, connection=conn):
    """Inserts a new entry in job_table and returns its id."""
    insert = job_table.insert().values(company_id=company_id, 
                                       type_=JobType.from_string(type_str).value, 
                                       title=title, description=description, how_to_apply=how_to_apply) 
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

In [12]:
def insert_location(job_id, description, connection=conn):
    """Inserts a new entry in location_table and returns its job_id."""
    insert = location_table.insert().values(job_id=job_id, description=description, 
                                       remote_ok=int('remote' in description.lower() 
                                                     or 'telecommute' in description.lower())) 
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

In [13]:
import datetime
created_at_pattern = '%a %b %d %X %Z %Y'

def insert_github_metadata(job_id, github_id, url, created_at_str, connection=conn):
    """Inserts a new entry in github_metadata_table and returns its job_id."""
    insert = github_metadata_table.insert().values(job_id=job_id, github_id=github_id, url=url,
                                       created_at=datetime.datetime.strptime(created_at_str, created_at_pattern))
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

In [14]:
from sqlalchemy.sql import select

def select_or_insert_company(name, url, logo_url, connection=conn):
    """Returns the company id either by selecting from or inserting in company_table"""
    s = select([company_table]).where(company_table.c.url == url)
    result = connection.execute(s).fetchone()
    if result is not None:
        return result['id']
    else:
        new_id = insert_company(name, url, logo_url, connection)
        return new_id

In [15]:
def select_or_insert_job(company_name, company_url, company_logo_url, 
                         github_id, github_url, github_created_at_str,
                         type_str, title, description, how_to_apply,
                         location_description):
    """
    Returns the job id either by selecting from or inserting in job_table.
    
    This function might also insert new location, github_metadata and company entries.
    """
    s = select([github_metadata_table]).where(github_metadata_table.c.github_id == github_id)
    result = conn.execute(s).fetchone()
    if result is not None:
        return result['job_id']
    else:
        with engine.begin() as transaction:
            company_id = select_or_insert_company(company_name, company_url, company_logo_url, transaction)
            job_id = insert_job(company_id, type_str, title, description, how_to_apply, transaction)
            insert_github_metadata(job_id, github_id, github_url, github_created_at_str, transaction)
            insert_location(job_id, location_description, transaction)
            
        return job_id

In [16]:
for job in json_response:
    select_or_insert_job(
        job['company'],
        job['company_url'],
        job['company_logo'],
        job['id'],
        job['url'],
        job['created_at'],
        job['type'],
        job['title'],
        job['description'],
        job['how_to_apply'],
        job['location']
    )

2017-01-06 11:50:49,551 INFO sqlalchemy.engine.base.Engine SELECT github_metadata.job_id, github_metadata.github_id, github_metadata.url, github_metadata.created_at 
FROM github_metadata 
WHERE github_metadata.github_id = ?
2017-01-06 11:50:49,552 INFO sqlalchemy.engine.base.Engine ('806cba3a-d38a-11e6-9b37-19faf2d9fe5b',)
2017-01-06 11:50:49,554 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-01-06 11:50:49,555 INFO sqlalchemy.engine.base.Engine SELECT company.id, company.name, company.url, company.logo_url 
FROM company 
WHERE company.url = ?
2017-01-06 11:50:49,556 INFO sqlalchemy.engine.base.Engine ('http://www.leadpages.net/',)
2017-01-06 11:50:49,558 INFO sqlalchemy.engine.base.Engine INSERT INTO company (name, url, logo_url) VALUES (?, ?, ?)
2017-01-06 11:50:49,559 INFO sqlalchemy.engine.base.Engine ('Leadpages', 'http://www.leadpages.net/', 'http://github-jobs.s3.amazonaws.com/012e2336-d389-11e6-867c-d25a8523294d.png')
2017-01-06 11:50:49,561 INFO sqlalchemy.engine.bas

In [17]:
from sqlalchemy.sql import text

s = text('SELECT * FROM company')
result = conn.execute(s)

for row in result:
    print('\n {}'.format(row))

2017-01-06 11:51:20,852 INFO sqlalchemy.engine.base.Engine SELECT * FROM company
2017-01-06 11:51:20,854 INFO sqlalchemy.engine.base.Engine ()

 (1, 'Leadpages', 'http://www.leadpages.net/', 'http://github-jobs.s3.amazonaws.com/012e2336-d389-11e6-867c-d25a8523294d.png')

 (2, 'University of California, Berkeley', 'http://www.berkeley.edu', 'http://github-jobs.s3.amazonaws.com/7a1dae50-d37b-11e6-8a62-e48674991ce0.png')

 (3, 'HHMI Janelia Research Campus', 'https://www.janelia.org/', 'http://github-jobs.s3.amazonaws.com/81a49ffc-d291-11e6-9919-6d19904950ce.jpg')

 (4, 'Brinker International', 'http://www.brinkerjobs.com', 'http://github-jobs.s3.amazonaws.com/1c00c4b6-d1c2-11e6-9d77-70a0c115d478.png')

 (5, 'Paf', 'https://www.paf.com/about-us', 'http://github-jobs.s3.amazonaws.com/71c4e220-d0f9-11e6-982f-8ee5a9eb085a.png')

 (6, 'Gram Games', 'http://gram.gs/', None)

 (7, 'Enza Zaden Beheer B.V.', 'http://www.enzazaden.nl/aboutus/', 'http://github-jobs.s3.amazonaws.com/af433b70-c914-11

In [18]:
s = text('SELECT * FROM job')
result = conn.execute(s)

for row in result:
    print('\n {}'.format(row))

2017-01-06 11:51:33,062 INFO sqlalchemy.engine.base.Engine SELECT * FROM job
2017-01-06 11:51:33,064 INFO sqlalchemy.engine.base.Engine ()

 (1, 1, 1, 'Cloud Engineer / Site Reliability Engineer', '<p>We promise you haven’t worked with anyone <a href="http://www.leadpages.net/careers">like us</a> before.  We’re a Minneapolis / St. Paul, MN based ... (6330 characters truncated) ... pply/YN38C7/Cloud-Engineer-Site-Reliability-Engineer?source=Github">HERE</a>. Thanks for checking us out - we look forward to hearing from you!</p>\n', '<p>To apply, please submit your tailored resume, a link to your LinkedIn profile, and a cover letter explaining why you feel you are a match for this ... (49 characters truncated) ... pply/YN38C7/Cloud-Engineer-Site-Reliability-Engineer?source=Github">HERE</a>. Thanks for checking us out - we look forward to hearing from you!</p>\n')

 (2, 2, 1, 'Data Services Librarian', '<p>POSITION ANNOUNCEMENT\nUNIVERSITY OF CALIFORNIA, BERKELEY</p>\n\n<p>Data Services Li

In [19]:
s = text('SELECT * FROM location')
result = conn.execute(s)

for row in result:
    print('\n {}'.format(row))

2017-01-06 11:51:47,957 INFO sqlalchemy.engine.base.Engine SELECT * FROM location
2017-01-06 11:51:47,959 INFO sqlalchemy.engine.base.Engine ()

 (1, 'Minneapolis, MN', 0)

 (2, 'Berkeley, CA 94720', 0)

 (3, 'Ashburn, VA', 0)

 (4, 'Dallas, Texas', 0)

 (5, 'Helsinki, Finland', 0)

 (6, 'London, UK ', 0)

 (7, 'London, UK ', 0)

 (8, 'Enkhuizen', 0)

 (9, 'Amsterdam', 0)

 (10, 'Eindhoven', 0)

 (11, 'Veldhoven', 0)

 (12, 'Los Angeles, CA', 0)

 (13, 'Bratislava', 0)

 (14, 'Radnor', 0)

 (15, 'Cambridge, MA', 0)

 (16, 'San Francisco or Remote', 1)

 (17, 'New York, NY', 0)

 (18, 'Zurich', 0)

 (19, 'Tulsa, OK, USA', 0)

 (20, 'Tulsa, OK, USA', 0)

 (21, 'San Francisco', 0)

 (22, 'Palo Alto, CA', 0)


In [20]:
s = text('SELECT * FROM github_metadata')
result = conn.execute(s)

for row in result:
    print('\n {}'.format(row))

2017-01-06 11:52:01,307 INFO sqlalchemy.engine.base.Engine SELECT * FROM github_metadata
2017-01-06 11:52:01,311 INFO sqlalchemy.engine.base.Engine ()

 (1, '806cba3a-d38a-11e6-9b37-19faf2d9fe5b', 'http://jobs.github.com/positions/806cba3a-d38a-11e6-9b37-19faf2d9fe5b', '2017-01-05 21:04:49.000000')

 (2, '812add76-d37b-11e6-8065-d8a87983108a', 'http://jobs.github.com/positions/812add76-d37b-11e6-8065-d8a87983108a', '2017-01-05 19:17:40.000000')

 (3, 'cdb81090-d291-11e6-9fc6-230a387788dd', 'http://jobs.github.com/positions/cdb81090-d291-11e6-9fc6-230a387788dd', '2017-01-04 16:32:23.000000')

 (4, '4847ca1a-d1c2-11e6-87c7-f7105d23b2fa', 'http://jobs.github.com/positions/4847ca1a-d1c2-11e6-87c7-f7105d23b2fa', '2017-01-03 14:39:33.000000')

 (5, '98f2edec-d0f9-11e6-8897-553ed26979e3', 'http://jobs.github.com/positions/98f2edec-d0f9-11e6-8897-553ed26979e3', '2017-01-02 14:42:36.000000')

 (6, 'e5ed5520-ce19-11e6-91ed-8e7aa8839a6b', 'http://jobs.github.com/positions/e5ed5520-ce19-11e6-91ed-

## Assignment
- Store the data you got from the API request in the database.
- Make sure you create a transaction (i.e., if an error occurs while saving a job's location data the database should rollback on all statements regarding that job).

## Assignment
- Execute simple SQL queries in each table and make sure everything looks as was intended.
- Create queries the following queries:
    - Find out how many jobs have the phrase "Data Science" and how many have the word "Python" in the description
    - Find out how many were created in the past 7 days
    - Find out how are "Remote Ok" and which companies those are with

In [22]:
s = text("SELECT COUNT() "
         "FROM job")
total, = conn.execute(s).fetchone()

s = text("SELECT COUNT() "
         "FROM job "
         "WHERE description LIKE '%python%'")
python, = conn.execute(s).fetchone()

print("\nThere are {} jobs with the 'Data Science' term, and {}% of them have 'Python' in the description."
      .format(total, 100 * python // total))

from datetime import datetime, timedelta

s = text("SELECT COUNT() "
         "FROM github_metadata "
         "WHERE created_at >= :week_ago")
last_week, = conn.execute(s, week_ago=datetime.today() - timedelta(days=7)
                         ).fetchone()

print("\n{}% of them have been created in the past 7 days."
      .format(100 * last_week // total))

2017-01-06 11:54:05,963 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM job
2017-01-06 11:54:05,965 INFO sqlalchemy.engine.base.Engine ()
2017-01-06 11:54:05,967 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM job WHERE description LIKE '%python%'
2017-01-06 11:54:05,968 INFO sqlalchemy.engine.base.Engine ()

There are 22 jobs with the 'Data Science' term, and 54% of them have 'Python' in the description.
2017-01-06 11:54:05,971 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM github_metadata WHERE created_at >= ?
2017-01-06 11:54:05,971 INFO sqlalchemy.engine.base.Engine (datetime.datetime(2016, 12, 30, 11, 54, 5, 970889),)

22% of them have been created in the past 7 days.


In [23]:
s = text("SELECT COUNT() "
         "FROM job "
         "JOIN location ON job.id = location.job_id "
         "WHERE remote_ok = :true")
remote_jobs, = conn.execute(s, true=int(True)).fetchone()

if remote_jobs > 0:
    s = text("SELECT name "
             "FROM company "
             "JOIN job ON company.id = job.company_id "
             "JOIN location ON job.id = location.job_id "
             "WHERE location.remote_ok = :true")
    remote_companies = conn.execute(s, true=int(True)).fetchall()
    
    if len(remote_companies) == 1:
        message = '\n{} of those is remote ok. The company with the remote job is: {}'
    else:
        message = '\n{} of those are remote ok. The companies with remote jobs are: {}'
        
    print(message.format(remote_jobs, " ;".join(str(company) for company, in remote_companies)))

2017-01-06 11:54:24,816 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM job JOIN location ON job.id = location.job_id WHERE remote_ok = ?
2017-01-06 11:54:24,817 INFO sqlalchemy.engine.base.Engine (1,)
2017-01-06 11:54:24,819 INFO sqlalchemy.engine.base.Engine SELECT name FROM company JOIN job ON company.id = job.company_id JOIN location ON job.id = location.job_id WHERE location.remote_ok = ?
2017-01-06 11:54:24,820 INFO sqlalchemy.engine.base.Engine (1,)

1 of those is remote ok. The company with the remote job is: Kaggle


In [24]:
conn.close()